## Pseudocode and suggestions for pyiron_workflows

#### Extend .iter to macro programming

    wf.nodes = wf.create.my_path.MyNode(a=1, b=2).iter(element=['Al', 'Fe'])

This should be easy to achieve by taking all relevant info to perform the present definition, i.e.,

    self.nodes = Workflow.create.for_node(
        body_node_class=Workflow.create.my_path.MyNode, # extract class name from node
        iter_on="element",
        obj='Al',
        other=['Fe']
    )

This would be much more intuitive. During the workshop people were actually trying such a construct.    

### Some ideas/thoughts to control execution

#### Direct approach

    wf.nodes = wf.create.my_path.MyNode(a=1, b=2).server(cores=4).iter(element=['Al', 'Fe', 'Pb']).server(workers=3, queue='test')

Run 3 MyNode jobs simultaneously on 4 cores each, i.e., the queue test is assumed/told to provide 12 cores. This notation can be easily extended to include additional control variables (e.g. for controling the log level, gpu vs cpu), etc.    

#### Indirect approach via workflow

    wf = Workflow('my_workflow').server(cores=12, queue='test')

    # run interactively (no submission to queue)
    wf.interactive_node = wf.create.my_path.MyNode(a=1, b=2) 

    # run by submitting to queue (note: the number of workers is not needed, is obtained from 12 cores/ 4 cores per node)
    wf.expensive_nodes = wf.create.my_path.MyNode(a=1, b=2).server(cores=4, to_queue=True).iter(element=['Al', 'Fe', 'Pb'])

    wf.run()

Alternative formulation (both could be supported)
    
    wf = Workflow('my_workflow')

    # run interactively (no submission to queue)
    wf.interactive_node = wf.create.my_path.MyNode(a=1, b=2) 

    # run by submitting to queue (note: the number of workers is not needed, it is obtained from 12 cores/ 4 cores per node)
    wf.expensive_nodes = wf.create.my_path.MyNode(a=1, b=2).server(cores=4, to_queue=True).iter(element=['Al', 'Fe', 'Pb'])

    wf.server(cores=12, queue='test').run()

### Further extensions of the server 

    server(
        hash_node = None, # take from node definition (default is False)
        log_level = wf.definitions.log_level.default # .debug, .production etc. via autocompletion
        
    )

Alternatively to 'server' we may consider adding additional objects that control hashing, output etc. to prevent overloading 'server' with input arguments